# Really tho'... how many ingredients could there possibly be???

In [1]:
# imports.  you're prolly gonna use all these, right?
import pdb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

import re
import string

In [2]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [2]:
# Pulling in recipe lists
df = pd.read_csv('../data/clean_recipes.csv', sep=';')
kgl1_df = pd.read_json('../data/kgl_ingredient_train.json')
kgl2_df = pd.read_json('../data/kgl_ingredient_test.json')

In [3]:
df = df.rename(columns={'Ingredients':'ingredients'})
df.head(3)

,Recipe Name,Review Count,Recipe Photo,Author,Prepare Time,Cook Time,Total Time,ingredients,Directions,RecipeID
0,Golden Crescent Rolls Recipe,304,https://images.media-allrecipes.com/userphotos...,Mike A.,25 m,15 m,3 h 10 m,"yeast,water,white sugar,salt,egg,butter,flour,...","Dissolve yeast in warm water.**Stir in sugar, ...",7000
1,Poppy Seed Bread with Glaze Recipe,137,https://images.media-allrecipes.com/userphotos...,Christina Jun,15 m,1 h,1 h 20 m,"flour,salt,baking powder,poppy,butter,vegetabl...",'Preheat oven to 350 degrees F (175 degrees C)...,7001
2,Applesauce Bread I Recipe,124,https://images.media-allrecipes.com/userphotos...,GAF55,10 m,1 h 20 m,1 h 30 m,"flour,egg,white sugar,vegetable oil,applesauce...",Preheat oven to 350 degrees F (175 degrees C)....,7003


In [4]:
# df ingredients is a single string, splitting into a list of strings
df['ingredients'] = df['ingredients'].apply(lambda x: x.split(','))

In [5]:
l1 = df['ingredients']
l2 = kgl1_df['ingredients']
l3 = kgl2_df['ingredients']

ingr_series = l1.append([l2, l3], ignore_index=True)

In [6]:
ingr_series[:10]

0    [yeast, water, white sugar, salt, egg, butter,...
1    [flour, salt, baking powder, poppy, butter, ve...
2    [flour, egg, white sugar, vegetable oil, apple...
3    [flour, baking powder, baking soda, salt, cinn...
4    [oat, buttermilk, vegetable oil, egg, brown su...
5    [shortening, white sugar, salt, milk, egg, lem...
6    [shortening, water, brown sugar, yeast, water,...
7    [water, cottage cheese, vegetable oil, egg, fl...
8    [margarine, white sugar, egg, banana, coffee, ...
9    [cornmeal, milk, egg, bell pepper, onion, garl...
Name: ingredients, dtype: object

In [7]:
def scrub_ingredients(ingr_list):
    
    chtble = str.maketrans('', '', string.punctuation + '0123456789')
    
    # drop special characters and multiple spaces, then convert to lowercase
    clean_list = []
    for ingr_string in ingr_list:
        ingr_string = re.sub(r'\W\s+', ' ', ingr_string.strip().translate(chtble))
        clean_list.append(ingr_string.lower())

    # convert to lowercase & drop known measurement words
    dropwords = ['ounce', 'ounces', 'oz', 'lb', ' pound', 'the', 'a', 'an'
                 'package', 'packages', 'inches', 'and', 'for', 'as', 'is']

    outlist = []

    for i in clean_list:
        words = i.split()
        words = [word for word in words if word not in dropwords]
        i = ' '.join(words)
        outlist.append(i)
    
    return outlist

In [8]:
ingr_s = ingr_series.apply(scrub_ingredients)
ingr_s[:10]

0    [yeast, water, white sugar, salt, egg, butter,...
1    [flour, salt, baking powder, poppy, butter, ve...
2    [flour, egg, white sugar, vegetable oil, apple...
3    [flour, baking powder, baking soda, salt, cinn...
4    [oat, buttermilk, vegetable oil, egg, brown su...
5    [shortening, white sugar, salt, milk, egg, lem...
6    [shortening, water, brown sugar, yeast, water,...
7    [water, cottage cheese, vegetable oil, egg, fl...
8    [margarine, white sugar, egg, banana, coffee, ...
9    [cornmeal, milk, egg, bell pepper, onion, garl...
Name: ingredients, dtype: object

In [9]:
def ingredient_count(dfcolumn):

    # Create dictionary with word count
    d = dict()

    for l in dfcolumn:
        for i in l:
            if i in d:
                d[i] = d[i] + 1
            else:
                d[i] = 1

    return pd.DataFrame(d.items(), columns=['ingredient', 'count'])

In [10]:
# create df of unique ingredients and their number of occurances in recipe dfs
ingr_count_df = ingredient_count(ingr_s).sort_values('count', ascending=False).reset_index(drop=True)
ingr_count_df.head(15)

,ingredient,count
0,salt,27050
1,water,11899
2,garlic,11287
3,butter,10534
4,onions,10008
5,olive oil,9889
6,sugar,9319
7,garlic cloves,7772
8,pepper,7080
9,vegetable oil,6874


# The long way round...

In [ ]:
## Reset ##

# copy ingr df, and remove all ingredients with only a single obseration in the dataset
#       (hoping to purge typos and bizarre edge cases)

ingr_count_categories = ingr_count_df.copy()
ingr_count_categories = ingr_count_categories[ingr_count_categories['count'] >= 2]
ingr_count_categories['categories'] = np.empty((len(ingr_count_categories), 0)).tolist()

root_ingr_list = []


In [101]:
# make a list of remaining ingredients to begin categorization process
uncategorized_ingr = ingr_count_categories.loc[(ingr_count_categories['categories'].str.len() == 0), :]
working_ingr_list = list(uncategorized_ingr['ingredient'])
print(len(working_ingr_list), 'items in working_ingr_list')
print('\nSnapshot of df remaining uncategorized')
ingr_count_categories.head()

ValueError: Lengths must match to compare

In [154]:
# # Method for removing any single item from category list by word
# ingr_categories = ingr_categories.remove('avacado')

# # Method for removing any single item from db by index
# ingr_count_df_trimmed.drop()

## gameplan...

### variables -
- list of "unique ingredients" working through
- df with ingredients & their assigned category/categories
- list of defined categories

### steps -
- maintain a list of "unique" ingredients
  - **ingr_list**
- take the root/essence of the first 5 words in df ingr_count_categories
  - red potatoes >> potato
  - sea salt >> salt
  - boiled cabbage >> cabbage
  - (etc...)
- print a snapshot of the way those words appear in the ingr_list
  - potato >> [potatoes, baked potato, etc....]
- manually identify any outliers that don't fit the root word
  - ...potato chips, ...
- list.append root word to df.category wherever that substring correctly appears in the ingredient list
  - 93 ---- zucchini ---- 1279 ---- []
  - 94 ---- potatoes ---- 1272 ---- \[potato]
  - 95 ---- bay leaf ---- 1264 ---- []
  - ...
  - 3163 ---- vanilla pudding   ---- 1279 ---- []
  - 3164 ---- potato chips      ---- 1272 ---- \[]
  - 3165 ---- halibut steak     ---- 1264 ---- []


In [1710]:
root_ingr_list = sorted(list(set(ingr_count_categories.loc[(ingr_count_categories['categories'].str.len() > 0), :]['categories'])))
working_ingr_list = list(ingr_count_categories.loc[(ingr_count_categories['categories'].str.len() == 0), :]['ingredient'])

print(f'current number of "root" ingredient words: {len(root_ingr_list)}')
print(f'number of remaining ingredients in working_ingr_list: {len(working_ingr_list)} \n\nNext top 10:')
working_ingr_list[0:10]

current number of "root" ingredient words: 132
number of remaining ingredients in working_ingr_list: 3070 

Next top 10:


['green chilies',
 'hot sauce',
 'red pepper flakes',
 'sauce',
 'coconut',
 'red pepper',
 'mushrooms',
 'corn',
 'pecan',
 'mushroom']

In [1711]:
all_wrds_list = ' '
all_wrds_list = all_wrds_list.join(working_ingr_list)
d = dict()

for i in all_wrds_list.split():
    if i in d:
        d[i] = d[i] + 1
    else:
        d[i] = 1

ingr_word_count_df = pd.DataFrame(d.items(), columns=['word', 'count'])
ingr_word_count_df = ingr_word_count_df.sort_values('count', ascending=False).reset_index(drop=True)
ingr_word_count_df.head(10)

,word,count
0,sauce,88
1,dried,55
2,mix,49
3,chile,48
4,seasoning,48
5,frozen,45
6,red,45
7,paste,45
8,fresh,44
9,powder,40


In [1712]:
ingr_set

{'applesauce',
 'avocado',
 'bacon',
 'bake',
 'baking powder',
 'baking soda',
 'basil',
 'bay leaves',
 'bean',
 'beef',
 'beer',
 'bell pepper',
 'bread',
 'bread crumbs',
 'butter',
 'cake',
 'cake mix',
 'capers',
 'carrot',
 'cayenne',
 'celery',
 'cheese',
 'chicken',
 'chili peppers',
 'chili powder',
 'chocolate',
 'cilantro',
 'cinnamon',
 'clove',
 'cooking spray',
 'coriander',
 'corn starch',
 'corn syrup',
 'cornbread',
 'cornflake',
 'cornmeal',
 'cornstarch',
 'cream',
 'cream cheese',
 'crushed red pepper',
 'crust',
 'cucumber',
 'cumin',
 'curry paste',
 'curry powder',
 'dough',
 'egg',
 'eggplant',
 'fennel',
 'fish sauce',
 'flour',
 'garam masala',
 'garlic',
 'garlic powder',
 'ginger',
 'goat',
 'ground beef',
 'ham',
 'hen',
 'honey',
 'ice cream',
 'jalapeno',
 'juice',
 'ketchup',
 'lemon',
 'lemon juice',
 'lettuce',
 'lime',
 'margarine',
 'mayonaise',
 'mayonnaise',
 'milk',
 'mozzarella',
 'mustard',
 'nutmeg',
 'oat',
 'oil',
 'olive',
 'onion',
 'orang

In [1713]:
ingr0_wordlist[0:10]

['corn',
 'apple',
 'saffron threads',
 'pinenuts',
 'pie',
 'green chile',
 'fresh dill',
 'cauliflower',
 'green cabbage',
 'coconut']

In [1714]:
search_root_word = 'raisin'
new_root_category = ''

if new_root_category == '':
    new_root_category = search_root_word

In [1715]:
# make a view of applicable ingredients with their index numbers
trimmed_df = ingr_count_categories.loc[(ingr_count_categories['categories'].str.len() == 0), :]
subject_ingr_df = trimmed_df[trimmed_df['ingredient'].str.contains(search_root_word)]
for i in subject_ingr_df.index:
    print(f"{i}: {list(subject_ingr_df.loc[i, ['ingredient']])[0]}")

In [1716]:
# to manually copy/paste/edit what items we don't want included in the category
subject_ingr_indxs = list(subject_ingr_df.index)
print(subject_ingr_indxs)

[]


In [1717]:
# comb through indexes and remove inaccurate indexes for that "category"

# # method for replacing categories
# indx_trim = ingr_count_categories.loc[(ingr_count_categories.categories == 'corn meal'), :].index
# new_root_category = 'replacement val'

# if all returns were acceptable:
indx_trim = subject_ingr_indxs.copy()

# if list of indexes needs to be curated:
# indx_trim = [359, 581, 871, 909, 1151, 1273, 1474, 1480, 1567, 1858, 2047, 2199, 2497, 2543, 2619, 2650, 2801, 3038, 3139, 3217, 3313, 3629, 3840, 4001, 4125, 4304, 4308, 4450, 4584, 4650, 4807, 4870, 4874, 4882, 4998, 5136, 5334, 5406, 5512]

In [1718]:
# apply new category label to remaining indexes in ingr_count_categories dataframe
ingr_count_categories.loc[indx_trim, ['categories']] = new_root_category

# quick peek at the updated categoriezed df
ingr_count_categories[ingr_count_categories['categories'] == new_root_category].head(5)

,ingredient,count,categories
196,raisins,595,raisin
226,raisin,534,raisin
384,golden raisins,291,raisin


In [1719]:
# create new working_ingr_list with uncategorized ingredients and start over
uncategorized_ingr = ingr_count_categories.loc[(ingr_count_categories['categories'].str.len() == 0), :]
working_ingr_list = list(uncategorized_ingr['ingredient'])

In [1720]:
# save categorized df, just in case a crash happens!!!
ingr_count_categories.to_csv('categorized_ingr2.csv', index=False)

In [1721]:
# snapshot of work still to be done:
total_ingr_count = sum(ingr_count_categories['count'])
missing_root_ingr = ingr_count_categories.loc[(ingr_count_categories['categories'].str.len() == 0), :]
perc_missing_root = sum(missing_root_ingr['count'])/total_ingr_count

print(f'percent of total ingr count missing a "root ingr" value: {perc_missing_root}\n')
print(f'"unique" ingr remaining: {len(working_ingr_list)/len(ingr_count_categories)}')

percent of total ingr count missing a "root ingr" value: 0.18103970054410795

"unique" ingr remaining: 0.5550533357439884


In [1722]:
ingr_set = set(ingr_count_categories.loc[(ingr_count_categories['categories'].str.len() > 0)].categories)

In [1723]:
#  get a rough idea what the progress is ---
checking_df = ingr_count_categories.copy()
testing_df = ingr_s.apply(pd.Series)
testing_df = testing_df.rename(columns = lambda x: 'ingr_' + str(x))
testing_df.head(3)

,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,...,ingr_55,ingr_56,ingr_57,ingr_58,ingr_59,ingr_60,ingr_61,ingr_62,ingr_63,ingr_64
0,yeast,water,white sugar,salt,egg,butter,flour,butter,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,flour,salt,baking powder,poppy,butter,vegetable oil,egg,milk,white sugar,vanilla,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,flour,egg,white sugar,vegetable oil,applesauce,raisin,cinnamon,baking soda,baking powder,sour cream,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1307]:
# testing_df[testing_df['ingr_0'] == 'bake']

In [1724]:
# create a dictionary pairing original values and their root ingr value
ingr_keys = list(ingr_count_categories['ingredient'])
ingr_vals = list(ingr_count_categories['categories'])

ingr_dict = {ingr_keys[i]: ingr_vals[i] for i in range(len(ingr_keys))} 

In [1725]:
# replace values in modeling df with replace vals
#   *** mapping dict vals is only available with pd.Series, stack >> map >> unstack as workaround
testing_sub_df = testing_df.copy()
testing_sub_df = testing_sub_df.stack().map(ingr_dict).unstack()
testing_sub_df = testing_sub_df.stack().apply(lambda y: np.nan if len(y)==0 else y).unstack()
testing_sub_df.head()

,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,...,ingr_55,ingr_56,ingr_57,ingr_58,ingr_59,ingr_60,ingr_61,ingr_62,ingr_63,ingr_64
0,yeast,water,sugar,salt,egg,butter,flour,butter,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,flour,salt,baking powder,poppyseed,butter,oil,egg,milk,sugar,vanilla,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,flour,egg,sugar,oil,applesauce,raisin,cinnamon,baking soda,baking powder,sour cream,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,flour,baking powder,baking soda,salt,cinnamon,nutmeg,sugar,oat,NaN,walnut,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,oat,milk,oil,egg,sugar,flour,baking powder,baking soda,salt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1730]:
testing_sub_df.isna().sum().head(25)

ingr_0     10892
ingr_1     11707
ingr_2     11843
ingr_3     12715
ingr_4     14498
ingr_5     17364
ingr_6     21260
ingr_7     25944
ingr_8     30589
ingr_9     35496
ingr_10    40293
ingr_11    44611
ingr_12    48302
ingr_13    51431
ingr_14    53972
ingr_15    56008
ingr_16    57697
ingr_17    58941
ingr_18    59905
ingr_19    60554
ingr_20    61051
ingr_21    61353
ingr_22    61589
ingr_23    61729
ingr_24    61824
dtype: int64

In [1726]:
ingr_0_list = testing_df.iloc[testing_sub_df[testing_sub_df['ingr_0'].isna()].index]['ingr_0']
ingr_0_list

77               berry
78              cereal
80               apple
105         baking mix
118          blueberry
             ...      
62047      red lentils
62050    raspberry jam
62059    yellow squash
62063      cauliflower
62065        hot sauce
Name: ingr_0, Length: 10892, dtype: object

In [1727]:
ingr_0_list = testing_df.iloc[testing_sub_df[testing_sub_df['ingr_0'].isna()].index]['ingr_0']

d = dict()

for i in ingr_0_list:
    if i in d:
        d[i] = d[i] + 1
    else:
        d[i] = 1

ingr0_word_df = pd.DataFrame(d.items(), columns=['word', 'count'])
ingr0_word_df = ingr0_word_df.sort_values('count', ascending=False).reset_index(drop=True)
ingr0_wordlist = list(ingr0_word_df['word'])
ingr0_word_df.head(15)

,word,count
0,corn,122
1,apple,97
2,saffron threads,90
3,pinenuts,89
4,pie,82
5,green chile,81
6,fresh dill,79
7,cauliflower,75
8,green cabbage,73
9,coconut,72


In [207]:
modeling_df = ingr_s.apply(pd.Series)
modeling_df = modeling_df.rename(columns = lambda x: 'ingr_' + str(x))
modeling_df.head(10)

,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,...,ingr_55,ingr_56,ingr_57,ingr_58,ingr_59,ingr_60,ingr_61,ingr_62,ingr_63,ingr_64
0,yeast,water,white sugar,salt,egg,butter,flour,butter,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,flour,salt,baking powder,poppy,butter,vegetable oil,egg,milk,white sugar,vanilla,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,flour,egg,white sugar,vegetable oil,applesauce,raisin,cinnamon,baking soda,baking powder,sour cream,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,flour,baking powder,baking soda,salt,cinnamon,nutmeg,brown sugar,oat,apple,walnut,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,oat,buttermilk,vegetable oil,egg,brown sugar,flour,baking powder,baking soda,salt,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,shortening,white sugar,salt,milk,egg,lemon,yeast,flour,fruit,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,shortening,water,brown sugar,yeast,water,white sugar,salt,bread,whole wheat,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,water,cottage cheese,vegetable oil,egg,flour,white sugar,baking soda,salt,yeast,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,margarine,white sugar,egg,banana,coffee,water,vanilla,flour,salt,baking powder,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,cornmeal,milk,egg,bell pepper,onion,garlic,salt,baking soda,white sugar,corn,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [208]:
(modeling_df.isnull().mean() * 100).head(25)

ingr_0      0.000000
ingr_1      0.069278
ingr_2      0.667000
ingr_3      2.571332
ingr_4      6.173774
ingr_5     11.951216
ingr_6     19.618489
ingr_7     28.648762
ingr_8     38.076979
ingr_9     47.830640
ingr_10    57.033302
ingr_11    65.470686
ingr_12    72.807682
ingr_13    78.997567
ingr_14    84.048398
ingr_15    87.987562
ingr_16    91.190449
ingr_17    93.689281
ingr_18    95.559780
ingr_19    96.872835
ingr_20    97.902334
ingr_21    98.538723
ingr_22    98.980167
ingr_23    99.284667
ingr_24    99.487667
dtype: float64

In [209]:
modeling_df = modeling_df.replace(np.nan, '', regex=True)
modeling_df.head()

,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,...,ingr_55,ingr_56,ingr_57,ingr_58,ingr_59,ingr_60,ingr_61,ingr_62,ingr_63,ingr_64
0,yeast,water,white sugar,salt,egg,butter,flour,butter,,,...,,,,,,,,,,
1,flour,salt,baking powder,poppy,butter,vegetable oil,egg,milk,white sugar,vanilla,...,,,,,,,,,,
2,flour,egg,white sugar,vegetable oil,applesauce,raisin,cinnamon,baking soda,baking powder,sour cream,...,,,,,,,,,,
3,flour,baking powder,baking soda,salt,cinnamon,nutmeg,brown sugar,oat,apple,walnut,...,,,,,,,,,,
4,oat,buttermilk,vegetable oil,egg,brown sugar,flour,baking powder,baking soda,salt,,...,,,,,,,,,,


In [210]:
ingr_int = {ingr:i for i, ingr in enumerate(ingr_count_df['ingredient'])}
int_ingr = {i:ingr for i, ingr in enumerate(ingr_count_df['ingredient'])}

In [211]:
# convert ingredients to numeric representations
encoded_df = modeling_df.applymap(ingr_int.get)
print(encoded_df.shape)
encoded_df.head()

(62069, 65)


,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,...,ingr_55,ingr_56,ingr_57,ingr_58,ingr_59,ingr_60,ingr_61,ingr_62,ingr_63,ingr_64
0,352,1,10,0,15,3,12,3,4232,4232,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
1,12,0,27,1089,3,9,15,14,10,25,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
2,12,15,10,9,679,226,44,35,27,31,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
3,12,27,35,0,44,131,24,278,253,122,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
4,278,90,9,15,24,12,27,35,0,4232,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232


In [212]:
# drop all recipes that have fewer than 5 ingredients
encoded_df = encoded_df[encoded_df['ingr_5'] < 4232]
encoded_df.reset_index(drop=True)
print(encoded_df.shape)
encoded_df.head()

(54227, 65)


,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,...,ingr_55,ingr_56,ingr_57,ingr_58,ingr_59,ingr_60,ingr_61,ingr_62,ingr_63,ingr_64
0,352,1,10,0,15,3,12,3,4232,4232,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
1,12,0,27,1089,3,9,15,14,10,25,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
2,12,15,10,9,679,226,44,35,27,31,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
3,12,27,35,0,44,131,24,278,253,122,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
4,278,90,9,15,24,12,27,35,0,4232,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232


In [213]:
derp_df = encoded_df.iloc[0:5]
derp_df

,ingr_0,ingr_1,ingr_2,ingr_3,ingr_4,ingr_5,ingr_6,ingr_7,ingr_8,ingr_9,...,ingr_55,ingr_56,ingr_57,ingr_58,ingr_59,ingr_60,ingr_61,ingr_62,ingr_63,ingr_64
0,352,1,10,0,15,3,12,3,4232,4232,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
1,12,0,27,1089,3,9,15,14,10,25,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
2,12,15,10,9,679,226,44,35,27,31,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
3,12,27,35,0,44,131,24,278,253,122,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232
4,278,90,9,15,24,12,27,35,0,4232,...,4232,4232,4232,4232,4232,4232,4232,4232,4232,4232


In [214]:
encoded_df = encoded_df[encoded_df['ingr_59'] == 4232]
print(encoded_df.shape)

(54226, 65)


In [215]:
def create_rec_sequences(recipe_row):

    # take length of row & create length 5 lists that can be added to sequences[]
    
    counter = 0

    r_list = []
    next_ingr = []
    
    for counter in range(0, len(recipe_row), 3):
        if recipe_row[counter + 5] == 4232:
            return r_list, next_ingr
        r_list.append(recipe_row[counter:counter+5])
        next_ingr.append(recipe_row[counter + 5])

In [216]:
#  create list of sequences for 5 item lists
#  i.e. sequences[0] = [352, 1, 10, 0, 15]
#       sequences[1] = [1, 10, 0, 15, 3]

# testing grounds!
sequences = []
next_ingr = []

for row in range(0, len(derp_df)):
    derpen = derp_df.iloc[row]

    seq_to_add, next_to_add = create_rec_sequences(derpen)
    
    for seq in seq_to_add:
        sequences.append(seq)
    
    for nxt in next_to_add:
        next_ingr.append(nxt)
        

print(sequences, next_ingr)

[ingr_0    352
ingr_1      1
ingr_2     10
ingr_3      0
ingr_4     15
Name: 0, dtype: int64, ingr_0      12
ingr_1       0
ingr_2      27
ingr_3    1089
ingr_4       3
Name: 1, dtype: int64, ingr_3    1089
ingr_4       3
ingr_5       9
ingr_6      15
ingr_7      14
Name: 1, dtype: int64, ingr_6      15
ingr_7      14
ingr_8      10
ingr_9      25
ingr_10    204
Name: 1, dtype: int64, ingr_9      25
ingr_10    204
ingr_11    146
ingr_12      3
ingr_13    204
Name: 1, dtype: int64, ingr_0     12
ingr_1     15
ingr_2     10
ingr_3      9
ingr_4    679
Name: 2, dtype: int64, ingr_3      9
ingr_4    679
ingr_5    226
ingr_6     44
ingr_7     35
Name: 2, dtype: int64, ingr_0    12
ingr_1    27
ingr_2    35
ingr_3     0
ingr_4    44
Name: 3, dtype: int64, ingr_3      0
ingr_4     44
ingr_5    131
ingr_6     24
ingr_7    278
Name: 3, dtype: int64, ingr_6      24
ingr_7     278
ingr_8     253
ingr_9     122
ingr_10    226
Name: 3, dtype: int64, ingr_0    278
ingr_1     90
ingr_2      9
ingr_3 

In [217]:
sequences = []
next_ingr = []

for row in range(0, len(encoded_df)):
    
    ingr_list = encoded_df.iloc[row]
    seq_to_add, next_to_add = create_rec_sequences(ingr_list)

    for seq in seq_to_add:
        sequences.append(seq)
    for nxt in next_to_add:
        next_ingr.append(nxt)

for i in range(0, 10):
    print(sequences[i], '\n', next_ingr[i])


ingr_0    352
ingr_1      1
ingr_2     10
ingr_3      0
ingr_4     15
Name: 0, dtype: int64 
 3
ingr_0      12
ingr_1       0
ingr_2      27
ingr_3    1089
ingr_4       3
Name: 1, dtype: int64 
 9
ingr_3    1089
ingr_4       3
ingr_5       9
ingr_6      15
ingr_7      14
Name: 1, dtype: int64 
 10
ingr_6      15
ingr_7      14
ingr_8      10
ingr_9      25
ingr_10    204
Name: 1, dtype: int64 
 146
ingr_9      25
ingr_10    204
ingr_11    146
ingr_12      3
ingr_13    204
Name: 1, dtype: int64 
 25
ingr_0     12
ingr_1     15
ingr_2     10
ingr_3      9
ingr_4    679
Name: 2, dtype: int64 
 226
ingr_3      9
ingr_4    679
ingr_5    226
ingr_6     44
ingr_7     35
Name: 2, dtype: int64 
 27
ingr_0    12
ingr_1    27
ingr_2    35
ingr_3     0
ingr_4    44
Name: 3, dtype: int64 
 131
ingr_3      0
ingr_4     44
ingr_5    131
ingr_6     24
ingr_7    278
Name: 3, dtype: int64 
 253
ingr_6      24
ingr_7     278
ingr_8     253
ingr_9     122
ingr_10    226
Name: 3, dtype: int64 
 15


In [218]:
print('\n sequences:', len(sequences), '\n', 'next_ingredients:', len(next_ingr))


 sequences: 129423 
 next_ingredients: 129423


In [219]:
max_pred_length = 10

In [220]:
x = np.zeros((len(sequences), max_pred_length, len(ingr_count_df)), dtype=np.bool)
y = np.zeros((len(sequences), len(ingr_count_df)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, ingr in enumerate(sequence):
        x[i,t,ingr] = 1
    
    y[i, next_ingr[i]] = 1

In [221]:
print(x.shape)
print(y.shape)

(129423, 10, 7631)
(129423, 7631)


In [222]:
model = Sequential()
model.add(LSTM(128, input_shape=(max_pred_length, len(ingr_count_df))))
model.add(Dense(len(ingr_count_df), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='nadam')

In [223]:
model.fit(x, y,
          batch_size=128,
          epochs=5)

Train on 129423 samples
Epoch 1/5
129423/129423 [==============================] - 421s 3ms/sample - loss: 6.4722
Epoch 2/5
129423/129423 [==============================] - 419s 3ms/sample - loss: 6.0842
Epoch 3/5
129423/129423 [==============================] - 432s 3ms/sample - loss: 5.8086
Epoch 4/5
129423/129423 [==============================] - 444s 3ms/sample - loss: 5.6051
Epoch 5/5
129423/129423 [==============================] - 439s 3ms/sample - loss: 5.4651


In [224]:
model.save('ingr_pred_model.pkl')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ingr_pred_model.pkl\assets


In [225]:
import json
with open('ingr_int.json', 'w') as fp:
    json.dump(ingr_int, fp)

with open('int_ingr.json', 'w') as fp:
    json.dump(int_ingr, fp)

In [30]:
ingr_names = [ingr_int.keys()]

In [31]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [33]:
known_ingr = ['peanut butter', 'flour', 'sugar', 'eggs', 'chocolate chips']

pred_next = []

for diversity in [0.2, 0.5, 1.0, 1.2]:

    start_ingr = [ingr_int[x] for x in known_ingr]
    
    for i in range(10):
        x_pred = np.zeros((1, max_pred_length, len(ingr_count_df)))
        for t, ingr in enumerate(start_ingr):
            x_pred[0, t, ingr] = 1

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_ingredient = int_ingr[next_index]

        pred_next.append(next_ingredient)


print(set(pred_next))

{'vanilla extract', 'sugar', 'active dry yeast', 'hot water', 'coconut milk', 'baking powder', 'yellow chives', 'white sugar', 'flour', 'cream cheese', 'water', 'salt', 'milk', 'steak sauce', 'carrot', 'nutmeg', 'unsalted butter', 'lemon juice', 'egg', 'baking soda'}


In [226]:
len(ingr_count_df)

7631

In [54]:
known_ingr = ['thyme', 'pepper', 'chicken']

pred_next = []

for diversity in [0.2, 0.5, 1.0, 1.2]:

    start_ingr = [ingr_int[x] for x in known_ingr]
    
    for i in range(8):
        x_pred = np.zeros((1, max_pred_length, len(ingr_count_df)))
        for t, ingr in enumerate(start_ingr):
            x_pred[0, t, ingr] = 1

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_ingredient = int_ingr[next_index]

        pred_next.append(next_ingredient)


print(set(pred_next))

{'green onion', 'pepper', 'basil', 'canola oil', 'allpurpose flour', 'tomato sauce', 'celery', 'peach preserves', 'tumeric', 'frosting', 'worcestershire sauce', 'ground cumin', 'bok choy', 'salt', 'chicken broth', 'saffron', 'oregano', 'sweet onion', 'pinto beans', 'preserved lemon'}


In [42]:
ingr_count_df[ingr_count_df['ingredient'] == 'galangal']

,ingredient,count
797,galangal,104


In [ ]:

test == FALSE

for 



assert requests.get(image_uri)














